In [ ]:
# 1) Imports & setup
import os, pandas as pd, numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
load_dotenv()
engine = create_engine(os.getenv("POSTGRES_URL"))
%matplotlib inline

In [ ]:
# 2) Basic row counts & freshness
qc_sql = """
select 'drivers' tbl, count(*) cnt from raw.drivers union all
select 'constructors', count(*) from raw.constructors union all
select 'races', count(*) from raw.races union all
select 'results', count(*) from raw.results union all
select 'pitstops', count(*) from raw.pitstops union all
select 'qualifying', count(*) from raw.qualifying;
"""
qc = pd.read_sql(text(qc_sql), engine)
display(qc)

In [ ]:
# 3) Missingness & distributions in core tables
res = pd.read_sql(text("select * from raw.results"), engine)
print(res.isna().mean().sort_values(ascending=False).head(12))
res[["grid","position","points"]].describe()

In [ ]:
# 4) Target definition sanity: is_top10
res["is_top10"] = res["position"].between(1,10)
res["is_top10"].value_counts(normalize=True)

In [ ]:
# 5) Season trend plots
by_season = pd.read_sql(text("""
select ra.season, count(*) n_results,
       avg(case when r.position between 1 and 10 then 1 else 0 end) top10_rate
from raw.results r join raw.races ra using (race_id)
group by 1 order by 1
"""), engine)
by_season.plot(x="season", y=["n_results","top10_rate"], subplots=False, figsize=(8,4), marker='o')
plt.title("Volume & Top10 rate over seasons"); plt.show()

In [ ]:
# 6) Qualifying vs finish correlation
qfin = pd.read_sql(text("""
select q.position as quali_pos, r.position as finish_pos
from raw.qualifying q join raw.results r using (race_id, driver_id)
where r.position is not null
"""), engine)
qfin.corr(numeric_only=True)
plt.scatter(qfin["quali_pos"], qfin["finish_pos"], s=8)
plt.xlabel("Quali position (lower=better)"); plt.ylabel("Finish position")
plt.title("Quali vs Finish"); plt.show()

In [ ]:
# 7) Pitstop distributions
pit = pd.read_sql(text("select duration_ms from raw.pitstops where duration_ms is not null"), engine)
pit["duration_s"] = pit["duration_ms"]/1000
pit["duration_s"].hist(bins=40, figsize=(7,4))
plt.title("Pit stop duration distribution (s)"); plt.show()

In [ ]:
# 8) Data joins preview for features view
feat_preview = pd.read_sql(text("""
select ra.season, ra.round, r.driver_id, r.grid,
       q.position as quali_pos
from raw.results r
join raw.races ra using (race_id)
left join raw.qualifying q using (race_id, driver_id)
limit 200
"""), engine)
feat_preview.head()

In [ ]:
# 9) Leakage checks quick pass (ensure no post-race info in features)
# Example: ensure 'time_ms' or 'status' are never used for pre-race features
sus = {"time_ms","status","fastest_lap"}
used_columns = {"grid","quali_pos"}  # expand as you add features
print("Suspicious columns present in features?", bool(sus & used_columns))

In [ ]:
# 10) Save lightweight PNGs for README/BI screenshots
out = "../bi/screenshots"
os.makedirs(out, exist_ok=True)
plt.figure(figsize=(6,3))
by_season.plot(x="season", y="top10_rate", legend=False)
plt.title("Top‑10 rate by season"); plt.tight_layout(); plt.savefig(f"{out}/top10_rate_by_season.png")